In [2]:
%cd ../

/home/qwj/code/HippoRAG


## `create_graph`


In [7]:
!python src/create_graph.py --help

usage: create_graph.py [-h] [--dataset DATASET] [--model_name MODEL_NAME]
                       [--extraction_model EXTRACTION_MODEL]
                       [--threshold THRESHOLD] [--save_facts_only]
                       [--create_graph] [--extraction_type EXTRACTION_TYPE]
                       [--cosine_sim_edges]

optional arguments:
  -h, --help            show this help message and exit
  --dataset DATASET
  --model_name MODEL_NAME
  --extraction_model EXTRACTION_MODEL
  --threshold THRESHOLD
  --save_facts_only
  --create_graph
  --extraction_type EXTRACTION_TYPE
  --cosine_sim_edges


### musique 

In [8]:
!python src/create_graph.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --extraction_type ner \
    --save_facts_only

100%|███████████████████████████████████| 11656/11656 [00:01<00:00, 9842.49it/s]
Correct Wiki Format: 0 out of 11656
100%|████████████████████████████████████| 1006/1006 [00:00<00:00, 66010.70it/s]


move files

In [9]:
import os
import shutil

def move_files(subfolder_name):
    # 定义新子文件夹的名称
    new_subfolder = os.path.join('output', subfolder_name)
    
    # 如果新子文件夹不存在，创建它
    if not os.path.exists(new_subfolder):
        os.makedirs(new_subfolder)

    # 获取文件夹下的所有文件（不包含子文件夹）
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
        
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if not (file_name.startswith("openie_") or file_name.startswith('.') or file_name.endswith("named_entity_output.tsv")):
                # 移动文件到新子文件夹
                print(f"Moved: {file_name}")
                shutil.move(file_path, os.path.join(new_subfolder, file_name))


In [10]:
move_files('musique_gpt')

Moved: musique_facts_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: .gitkeep
Moved: musique_facts_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Moved: query_to_kb.tsv
Moved: rel_kb_to_kb.tsv
Moved: kb_to_kb.tsv
Moved: musique_facts_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Moved: musique_facts_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
Moved: musique_facts_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
